## Multiple Cruises
You may want to gather data for all the cruises in a particular category: e.g., by a particular vessel, related to a particular topic, in a particular latitude range, etc. Here's some example code to start from.

In [ ]:
# # Local installation:
# import os
# package_root = os.path.join("", '..')
# %pip install -e "{package_root}"

In [ ]:
import os
from datetime import datetime as dt
import pandas as pd
import openspace_rvdata.r2r2df as r2r
import openspace_rvdata.tracks as trk

import logging
logger = logging.getLogger(__name__)

In [ ]:
url = r2r.get_r2r_url(vessel_name = "Revelle")
mdf = r2r.get_cruise_metadata(url)
# mdf = mdf.head(5) # first few rows only, for testing.

In [ ]:
for cruise_id in mdf.cruise_id:
    # todo: add check for if cruise_id is already covered in tmp directory
    is_in_filename = any(cruise_id in filename for filename in os.listdir("tmp/"))
    if is_in_filename:
        print("Got a file for " + cruise_id + "; moving on.")
        continue
    else:
        try:
            print("Pulling data for" + cruise_id)
            r2r.get_cruise_nav(cruise_id, "60min")
            
        except:
            print("Ran into an issue with " + cruise_id)
            continue

In [ ]:
from pathlib import Path
for file_path in Path("tmp").rglob('*_1min.geoCSV'):
    if file_path.is_file():
        fname = str(file_path.stem)
        trk.convert_geocsv_to_geojson(file_path, "tmp/"+fname+".geoJSON")

Adapted code below from https://gist.github.com/ericrobskyhuntley/790937a759fd89ed77c8831f880f854c#file-geojson-merge-py 

In [ ]:
import os
import glob
import geojson

json_dir_name = "../notebooks/tmp/"
json_pattern = os.path.join(json_dir_name,'*.geoJSON')
file_list = glob.glob(json_pattern)

collection = []

for file in file_list:
    with open(file) as f:
        layer = geojson.load(f)
        collection.append(layer)

geo_collection = geojson.GeometryCollection(collection)

with open('tmp/test_collection.geojson', 'w') as f:
    geojson.dump(geo_collection, f)

In [ ]:
trk.get_cruise_asset(mdf)
trk.get_cruise_keyframes("tmp/"+cruise_id+"_1min.geoCSV")

In [ ]:
json_dir_name = "../notebooks/tmp/"
json_pattern = os.path.join(json_dir_name,'*.geoJSON')
file_list = glob.glob(json_pattern)
file_list
# json_pattern